In [27]:
import pickle as pk
import numpy as np
from normalization import *
import torch
from dataloader import combine_sparse_tensors
from tqdm import tqdm
from datetime import datetime, timedelta

In [15]:
def data_scaler(data):
    data = torch.FloatTensor(data)
    scaler = StandardScaler(mean = data.mean(dim = 0), std = data.std(dim = 0))
    return scaler

In [19]:
interval = 4
reg_num = 490
treat_dim = 10
tim_num = 24
input_window = 24
output_window = 24
training_rate = 0.8
validate_rate = 0.1
causal = True

In [3]:
with open('/home/yangxiaojie/KDD2025/model/data/odmetrics_sparse_tensors.pk', 'rb') as f:
    adjacency_matrix = pk.load(f)

/home/yangxiaojie/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
flows = np.load('/home/yangxiaojie/KDD2025/model/data/flows.npy')
flows = flows[:, ::interval, :]
flows = flows.reshape(len(adjacency_matrix), reg_num)

In [5]:
treatment_path = '/home/yangxiaojie/KDD2025/samples・説明書/treatment_scores_0618.pk'
with open(treatment_path, 'rb') as f:
    treatments = pk.load(f)

In [6]:
treat_dict = {}
alltreat = []

alltreat = np.zeros((len(adjacency_matrix), reg_num, treat_dim))
alltreat[:, :, 3] = 100

for d in treatments:
    for t in treatments[d]:
        treat_dict[d * tim_num + t] = {}
        for c in treatments[d][t]:
            treat = np.mean([_ for _ in treatments[d][t][c] if len(_) == treat_dim], axis = 0)
            if np.isnan(treat).any():
                treat = np.array([0, 0, 0, 100, 0, 0, 0, 0, 0, 0])
            alltreat[d * tim_num + t, c] = treat

/home/yangxiaojie/anaconda3/envs/py38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/yangxiaojie/anaconda3/envs/py38/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
indices = []
for i in range(flows.shape[0]- input_window - output_window):
    indices.append(list(range(i, i + input_window + output_window)))

In [17]:
length = len(indices)
train_len = int(length * training_rate)
valid_len = int(length * validate_rate)
train_uplimit = indices[train_len][-1]
scaler = data_scaler(flows[:train_uplimit, :])

In [25]:
data = []
for index, indice in enumerate(tqdm(indices, desc="Processing")):
    x = scaler.transform(torch.FloatTensor(flows[indice[:input_window], :]))

    y = scaler.transform(torch.FloatTensor(flows[indice[input_window:], :]))
    if causal:
        treats = alltreat[indice[0] : indice[input_window], :, :]
    else:
        treats = None

    adj = combine_sparse_tensors(adjacency_matrix[indice[0] : indice[input_window]])
    indice = torch.LongTensor(indice)
    t = indice[:input_window] % tim_num  # 获取时间步
    t = t.unsqueeze(1).expand(-1, x.shape[1])  # 扩展时间步的维度

    data.append([indice, x, y, t, treats, adj])

Processing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8736/8736 [01:57<00:00, 74.46it/s]


In [ ]:
start_date = '2023-04-01'

In [32]:
data[0][0]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])

In [38]:
def calculate_hour(hours_passed, start_date = '2023-04-01'):
    # 将起始日期转换为 datetime 对象
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
    
    # 计算目标日期时间
    target_datetime = start_datetime + timedelta(hours=hours_passed)
    
    # 检查是否为周末
    is_weekend = target_datetime.weekday() >= 5  # 5 是周六，6 是周日
    
    # 计算目标小时（0-23 或 24-47）
    if is_weekend:
        result_hour = target_datetime.hour + 24
    else:
        result_hour = target_datetime.hour
    
    return result_hour

In [59]:
def calculate_week_hour(hours_passed, start_date = '2023-04-01'):
    # 将起始日期转换为 datetime 对象
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
    
    # 计算目标日期时间
    target_datetime = start_datetime + timedelta(hours=hours_passed)
    
    # 计算从周开始的小时数（0-143）
    #days_since_week_start = target_datetime.weekday() # 星期一为0
    result_hour = target_datetime.weekday() * 24 + target_datetime.hour
    
    return result_hour

In [70]:
start_date = '2023-04-01'
start_datetime = datetime.strptime(start_date, '%Y-%m-%d')

# 计算目标日期时间
target_datetime = start_datetime + timedelta(hours=20)

In [73]:
target_datetime.weekday()

5

In [72]:
target_datetime.weekday() * 24 + target_datetime.hour

140

In [60]:
[calculate_week_hour(i.numpy().tolist()) for i in data[0][0]]

[120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167]

In [45]:
data[0][0]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])